<a href="https://colab.research.google.com/github/jennif8r/LllForIFPR/blob/main/MapsWeb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Configuração do Ambiente


##2. Instalação de Bibliotecas

In [1]:
!pip install requests
!pip install beautifulsoup4


##3. Requisição e Análise do HTML


In [9]:
import requests
from bs4 import BeautifulSoup

# URL do site da universidade fictícia
url = 'https://ifpr.edu.br/colombo/'

# Fazer a requisição HTTP para obter o conteúdo da página
response = requests.get(url)

# Analisar o conteúdo HTML da página
soup = BeautifulSoup(response.content, 'html.parser')

# Exibir os primeiros 1000 caracteres do conteúdo HTML para verificar
print(soup.prettify()[:10000])


<!DOCTYPE html>
<html lang="pt-BR" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://opengraphprotocol.org/schema/">
 <head>
  <!-- Google tag (gtag.js) -->
  <script async="" src="https://www.googletagmanager.com/gtag/js?id=G-Z8DEPESPKF">
  </script>
  <script>
   window.dataLayer = window.dataLayer || [];

		function gtag() {
			dataLayer.push(arguments);
		}
		gtag('js', new Date());

		gtag('config', 'G-Z8DEPESPKF');
  </script>
  <!-- Facebook Pixel Code -->
  <script>
   ! function(f, b, e, v, n, t, s) {
			if (f.fbq) return;
			n = f.fbq = function() {
				n.callMethod ?
					n.callMethod.apply(n, arguments) : n.queue.push(arguments)
			};
			if (!f._fbq) f._fbq = n;
			n.push = n;
			n.loaded = !0;
			n.version = '2.0';
			n.queue = [];
			t = b.createElement(e);
			t.async = !0;
			t.src = v;
			s = b.getElementsByTagName(e)[0];
			s.parentNode.insertBefore(t, s)
		}(window, document, 'script',
			'https://connect.facebook.net/en_US/fbevents.js');
		fbq('init', '86

##4 - Pre-processamento


In [6]:
import requests
from bs4 import BeautifulSoup
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Baixar recursos necessários do NLTK
nltk.download('punkt')
nltk.download('stopwords')

# Função para limpar o texto
def clean_text(text):
    text = re.sub(r'\s+', ' ', text)  # Remover espaços extras
    text = re.sub(r'\[.*?\]', '', text)  # Remover texto entre colchetes
    text = re.sub(r'<.*?>', '', text)  # Remover HTML tags
    return text

# Função para tokenizar e remover stop words
def preprocess_text(text):
    stop_words = set(stopwords.words('portuguese'))
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word.lower() not in stop_words]
    return ' '.join(filtered_text)

# URL do site
url = 'https://ifpr.edu.br/colombo/'

# Fazer a requisição HTTP para obter o conteúdo da página
response = requests.get(url)

# Verificar se a requisição foi bem-sucedida
if response.status_code == 200:
    print("Requisição bem-sucedida!")
else:
    print("Erro na requisição:", response.status_code)

# Analisar o conteúdo HTML da página
soup = BeautifulSoup(response.content, 'html.parser')

# Remover o conteúdo do rodapé (footer) da página, se presente
footer = soup.find('footer')
if footer:
    footer.decompose()

# Remover a <div> especificada pela classe "conteudo-barra-brasil"
conteudo_barra_brasil = soup.find('div', class_='conteudo-barra-brasil')
if conteudo_barra_brasil:
    conteudo_barra_brasil.decompose()

# Extrair texto e links
content = []
for element in soup.find_all(['p', 'a', 'div', 'span', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6']):
    if element.name == 'a' and element.has_attr('href'):
        content.append(f"{element.get_text()} (Link: {element['href']})")
    else:
        content.append(element.get_text())

# Combinar todo o conteúdo em uma única string
page_content = '\n'.join(content)

# Aplicar limpeza de texto
cleaned_text = clean_text(page_content)

# Aplicar pré-processamento
preprocessed_text = preprocess_text(cleaned_text)

# Salvar o conteúdo em um arquivo de texto
with open('testDois.txt', 'w', encoding='utf-8') as file:
    file.write(preprocessed_text)

print("Pre-processamento completo. Conteúdo salvo em 'Index IFPR.txt'")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Requisição bem-sucedida!
Pre-processamento completo. Conteúdo salvo em 'Index IFPR.txt'


## 1. Preparar os Dados


In [16]:
from datasets import Dataset
import pandas as pd

# Carregar o texto pré-processado do arquivo
with open('testDois.txt', 'r', encoding='utf-8') as file:
    preprocessed_text = file.read()

# Dividir o texto em partes menores para não ultrapassar o limite de 512 tokens
def split_text(text, max_length=512):
    words = text.split()
    return [' '.join(words[i:i + max_length]) for i in range(0, len(words), max_length)]

# Aplicar a divisão
texts = split_text(preprocessed_text, max_length=512)

# Criar um DataFrame a partir dos textos divididos
data = {'text': texts}
df = pd.DataFrame(data)

# Criar um dataset a partir do DataFrame
dataset = Dataset.from_pandas(df)

print(df)


                                                text
0  Portal Governo Brasileiro Portal Governo Brasi...
1  Notícias Conselho Superior Instituto Federal P...
2  – IPVC Portugal Convocação matrícula vagas rem...
3  Seção Pedagógica Assuntos Estudantis SEPAE Equ...
4  Informações sobre vida funcional Página Servid...
5  integral participação programa pós-graduação s...
6  //ifpr.edu.br/colombo/convocacao-para-matricul...


##Configurar e Carregar o Modelo
Vamos usar o modelo bert-base-portuguese-cased da Neuralmind e a biblioteca transformers para carregar o modelo e o tokenizer.

In [17]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, DataCollatorForLanguageModeling, TrainingArguments, Trainer

# Carregar o tokenizer e o modelo pré-treinado
tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')
model = AutoModelForMaskedLM.from_pretrained('neuralmind/bert-base-portuguese-cased')

# Tokenizar o dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=512)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Preparar os rótulos de MLM
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True,
    mlm_probability=0.15
)

# Definir os argumentos de treinamento
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)


Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Map:   0%|          | 0/7 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


##3. Treinar o Modelo
Vamos treinar o modelo utilizando os dados tokenizados.

In [ ]:
# Criar o objeto Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,
)

# Iniciar o treinamento
trainer.train()
